# Imported Libararies

In [52]:
import random
import math
import time

# Minimax & Alpha Pruning on Tic-Tac-Toe

## Tic Tac Toe Game Board & Winner Rules

In [53]:
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]

    def print_board(self):
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        return ' ' in self.board

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            return True
        return False

    def winner(self, square, letter):
        row_ind = square // 3
        row = self.board[row_ind*3:(row_ind+1)*3]
        if all([spot == letter for spot in row]):
            return True

        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True

        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0,4,8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2,4,6]]
            if all([spot == letter for spot in diagonal2]):
                return True
        return False

## AI Playing with Minimax Algorithm

In [54]:
def minimax(state, player):
    max_player = 'X'
    other_player = 'O' if player == 'X' else 'X'

    if check_winner(state, other_player):
        return {'position': None, 'score': 1 * (len(state.available_moves()) + 1) if other_player == max_player else -1 * (len(state.available_moves()) + 1)}

    elif not state.empty_squares():
        return {'position': None, 'score': 0}

    if player == max_player:
        best = {'position': None, 'score': -math.inf}
    else:
        best = {'position': None, 'score': math.inf}

    for possible_move in state.available_moves():
        state.make_move(possible_move, player)
        sim_score = minimax(state, other_player)

        state.board[possible_move] = ' '
        sim_score['position'] = possible_move

        if player == max_player:
            if sim_score['score'] > best['score']:
                best = sim_score
        else:
            if sim_score['score'] < best['score']:
                best = sim_score

    return best

## AI Playing with Alpha-Beta Pruning Algorithm

In [55]:
def minimax_alpha_beta(state, player, alpha=-math.inf, beta=math.inf):
    max_player = 'X'
    other_player = 'O' if player == 'X' else 'X'

    if check_winner(state, other_player):
        return {'position': None, 'score': 1 * (len(state.available_moves()) + 1) if other_player == max_player else -1 * (len(state.available_moves()) + 1)}

    elif not state.empty_squares():
        return {'position': None, 'score': 0}

    if player == max_player:
        best = {'position': None, 'score': -math.inf}
    else:
        best = {'position': None, 'score': math.inf}

    for possible_move in state.available_moves():
        state.make_move(possible_move, player)
        sim_score = minimax_alpha_beta(state, other_player, alpha, beta)

        state.board[possible_move] = ' '
        sim_score['position'] = possible_move

        if player == max_player:
            if sim_score['score'] > best['score']:
                best = sim_score
            alpha = max(alpha, sim_score['score'])
        else:
            if sim_score['score'] < best['score']:
                best = sim_score
            beta = min(beta, sim_score['score'])

        if beta <= alpha:
            break

    return best

## Game Play

In [56]:
def check_winner(game, player):
    for move in range(9):
        if game.board[move] == player and game.winner(move, player):
            return True
    return False


def play_game(use_alpha_beta=False):
    game = TicTacToe()
    player_letter = random.choice(['O','X'])
    ai_letter = 'X' if player_letter == 'O' else 'O'

    current_letter = random.choice(['O', 'X'])
    print(f"\nYour Symbol: {player_letter}")
    print(f"AI Symbol: {ai_letter}")
    if current_letter == player_letter:
        print(f"\nYou go first!")
    else:
        print(f"\nAI goes first!")
    while game.empty_squares():
        if current_letter == ai_letter:
            print("\nAI's Turn:")
            start_time = time.time()
            if use_alpha_beta:
                move = minimax_alpha_beta(game, ai_letter)['position']
            else:
                move = minimax(game, ai_letter)['position']
            move_time = time.time() - start_time
            print(f"AI took {move_time:.6f} seconds to move.")
        else:
            print("\nYour Turn:")
            game.print_board()
            valid_move = False
            while not valid_move:
                try:
                    move = int(input('Choose your move (0-8): '))
                    if move not in game.available_moves():
                        raise ValueError
                    valid_move = True
                except ValueError:
                    print('Invalid move. Try again.')

        game.make_move(move, current_letter)
        game.print_board()

        if game.winner(move, current_letter):
            if current_letter == ai_letter:
                print("AI wins! 😈")
            else:
                print("You win! 🎉")
            return

        current_letter = 'O' if current_letter == 'X' else 'X'

    print("It's a tie! 🤝")

## Compare Performance of Both Algorithm

In [57]:
def compare_performance():
    game1 = TicTacToe()
    game2 = TicTacToe()

    print("\nComparing performance...")

    start_time = time.time()
    minimax(game1, 'X')
    minimax_time = time.time() - start_time

    start_time = time.time()
    minimax_alpha_beta(game2, 'X')
    alphabeta_time = time.time() - start_time

    print(f"\nMinimax took: {minimax_time:.6f} seconds")
    print(f"Alpha-Beta Pruning took: {alphabeta_time:.6f} seconds")
    if alphabeta_time < minimax_time:
        print("\n✅ Alpha-Beta Pruning is faster!")
    else:
        print("\n❌ Minimax was faster (weird!)")

## Menu to choose Desire Option

In [58]:
def menu():
    print("\nWelcome to Tic-Tac-Toe with AI!")
    while True:
        print("\n1. Play against AI (Minimax)")
        print("2. Play against AI (Alpha-Beta Pruning)")
        print("3. Compare performance")
        print("4. Exit")

        choice = input("Choose an option (1-4): ")

        if choice == '1':
            play_game(use_alpha_beta=False)
        elif choice == '2':
            play_game(use_alpha_beta=True)
        elif choice == '3':
            compare_performance()
        elif choice == '4':
            print("Bye! 👋")
            break
        else:
            print("Invalid choice, try again.")

## Run Below Function to Start Game

In [59]:
menu()


Welcome to Tic-Tac-Toe with AI!

1. Play against AI (Minimax)
2. Play against AI (Alpha-Beta Pruning)
3. Compare performance
4. Exit
Choose an option (1-4): 1

Your Symbol: X
AI Symbol: O

AI goes first!

AI's Turn:
AI took 4.955215 seconds to move.
| O |   |   |
|   |   |   |
|   |   |   |

Your Turn:
| O |   |   |
|   |   |   |
|   |   |   |
Choose your move (0-8): 4
| O |   |   |
|   | X |   |
|   |   |   |

AI's Turn:
AI took 0.065647 seconds to move.
| O | O |   |
|   | X |   |
|   |   |   |

Your Turn:
| O | O |   |
|   | X |   |
|   |   |   |
Choose your move (0-8): 2
| O | O | X |
|   | X |   |
|   |   |   |

AI's Turn:
AI took 0.001951 seconds to move.
| O | O | X |
|   | X |   |
| O |   |   |

Your Turn:
| O | O | X |
|   | X |   |
| O |   |   |
Choose your move (0-8): 3
| O | O | X |
| X | X |   |
| O |   |   |

AI's Turn:
AI took 0.000312 seconds to move.
| O | O | X |
| X | X | O |
| O |   |   |

Your Turn:
| O | O | X |
| X | X | O |
| O |   |   |
Choose your move (0-8):